In [164]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

products = pd.read_csv("products.csv")
sales = pd.read_csv("sales.csv")

In [165]:
products.head(2)

,productID,title,category,description
0,A001,taza,MEN,para beber
1,A002,vaso,WOMEN,para beber siendo mujer


In [166]:
sales.head(2)

,dia,mes,año,productID,importe
0,15,1,2019,A001,301
1,16,1,2019,A001,401


In [167]:
#  Indicar los títulos de los productos de la categoría "Men" para los cuales el Importe de
#  venta supera el promedio mensual de ventas de los productos de la misma categoría.

# Ventajas de usar transform, me deja la misma cantidad de filas que el anterior
# (repite resultado de cada groupby) y eso hace mas facil el join luego

ventas_con_categoria = pd.merge(products, sales, on = 'productID', how = 'left')
ventas_con_categoria.head(8)

,productID,title,category,description,dia,mes,año,importe
0,A001,taza,MEN,para beber,15.0,1.0,2019.0,301.0
1,A001,taza,MEN,para beber,16.0,1.0,2019.0,401.0
2,A001,taza,MEN,para beber,17.0,1.0,2019.0,101.0
3,A001,taza,MEN,para beber,20.0,1.0,2019.0,102.0
4,A001,taza,MEN,para beber,22.0,1.0,2019.0,202.0
5,A001,taza,MEN,para beber,24.0,1.0,2019.0,302.0
6,A001,taza,MEN,para beber,10.0,1.0,2019.0,101.0
7,A001,taza,MEN,para beber,11.0,1.0,2019.0,102.0


In [168]:
#Agrupo por categoria
grouped = ventas_con_categoria.groupby('category')

#Agrego columna de promedio mensual de ventas para cada categoria
ventas_con_categoria['avg_monthly_sale'] = grouped['importe'].transform(np.mean)

ventas_con_categoria.head()

,productID,title,category,description,dia,mes,año,importe,avg_monthly_sale
0,A001,taza,MEN,para beber,15.0,1.0,2019.0,301.0,176.454545
1,A001,taza,MEN,para beber,16.0,1.0,2019.0,401.0,176.454545
2,A001,taza,MEN,para beber,17.0,1.0,2019.0,101.0,176.454545
3,A001,taza,MEN,para beber,20.0,1.0,2019.0,102.0,176.454545
4,A001,taza,MEN,para beber,22.0,1.0,2019.0,202.0,176.454545


In [169]:
vcc = ventas_con_categoria

#Me quedo con la categoria MEN
men_sales = vcc.loc[vcc['category'] == 'MEN']

#Me quedo con los que superan el promedio mensual de ventas de hombre
men_over_avg = men_sales.loc[men_sales['importe'] > men_sales['avg_monthly_sale']]

pd.unique(men_over_avg['title'])

array(['taza'], dtype=object)

In [175]:
# B. Indicar el top-10 de productos que se vendieron mayor cantidad de días de forma
# consecutiva


# Primero paso el formato de fechas a un formato de fechas tipo datetime que me
# permita calcular el delta de tiempo en dias entre todas

df = sales

# Genero mi columnita de fechas hecha a partir de una funcion aplicada fila a fila
df['fecha'] = df.apply(lambda x: str(x['año']) + '-' + str(x['mes']) + '-' + str(x['dia']), axis = 1)

# Paso a datetime
df['fecha'] = pd.to_datetime(df['fecha'])

df.sort_values(['productID', 'fecha'], ascending=[True, True], inplace = True)

#df.groupby('productID')

# Genero columna de fecha anterior
df['fecha_ant'] = df['fecha'].shift(1)

# Genero delta en dias entre fecha y fecha anterior
df['delta'] = (df['fecha'] - df['fecha_ant']).dt.days

# Genero columna que indica 0 para consecutivos 
df['consec'] = df['delta'].apply(lambda x: 0 if x == 1 else 1)

# Acumulo consec, de manera que hay = consec_cum para fechas consecutivas
df['consec_cum'] = df.groupby('productID')['consec'].transform(np.cumsum)

# Agrupo de a productos, contabilizo la frecuencia de cada valor de consec_cum
# (pues este es el mismo en fechas consecutivas) y me quedo con el maximo
# para cada uno de ellos
products_max_consec = df.groupby('productID').apply(lambda x: x['consec_cum'].value_counts().max())

# Ordeno de mayor a menor
products_max_consec.sort_values(ascending = False, inplace = True)

#Me quedo con los 10 primeros
products_max_consec.head(10)

productID
A003    22
A001    19
A002     9
dtype: int64

In [148]:
#SOLUCION PREVIA QUE SOLO ADMITIA 1 UNICO PRODUCTO PARA ANALIZAR


# B. Indicar el top-10 de productos que se vendieron mayor cantidad de días de forma
# consecutiva


# Primero paso el formato de fechas a un formato de fechas tipo datetime que me
# permita calcular el delta de tiempo en dias entre todas

df = sales

# Genero mi columnita de fechas hecha a partir de una funcion aplicada fila a fila
df['fecha'] = df.apply(lambda x: str(x['año']) + '-' + str(x['mes']) + '-' + str(x['dia']), axis = 1)

# Paso a datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Genero columna de fecha anterior
df['fecha_ant'] = df['fecha'].shift(1)

# Genero delta en dias entre fecha y fecha anterior
df['delta'] = (df['fecha'] - df['fecha_ant']).dt.days

# Genero columna que indica 0 para consecutivos 
df['consec'] = df['delta'].apply(lambda x: 0 if x == 1 else 1)

# Acumulo consec, de manera que hay = consec_cum para fechas consecutivas
df['consec_cum'] = df['consec'].cumsum()

# Consigo el valor maximo de todos los valores de consec_cum
df['consec_cum'].value_counts().max()

19